# Script conjunto: Carga y optimización de datos en MySQL
Este notebook integra la carga de datos generales y la optimización de compresores en la base de datos `reto8_azul_claro`.

In [9]:
import pandas as pd
import numpy as np
import random
import json
import mysql.connector
from sqlalchemy import create_engine
import re
from pymongo import MongoClient

## 1. Cargar credenciales y definir conexión

In [10]:
# Puedes modificar aquí si quieres cargar desde archivo json
usuario = "root"
contrasena = "1234"
ruta = "127.0.0.1"
database = 'reto8_azul_claro'

## 2. Crear base de datos (si no existe)

In [11]:
conn = mysql.connector.connect(user=usuario, password=contrasena, host=ruta)
cur = conn.cursor()
cur.execute(f"CREATE DATABASE IF NOT EXISTS {database}")
conn.close()

## 3. Cargar y transformar datasets

In [12]:
# Compresores
compresores = pd.read_csv("../Datos/Transformados/df_comp.csv")

# Apache logs
pattern = r'(?P<client_ip>\S+) (?P<ident>\S+) (?P<auth>\S+) \[(?P<timestamp>[^\]]+)\] "(?P<http_method>\S+) (?P<request_path>\S+) HTTP/(?P<http_version>\S+)" (?P<http_status>\S+) (?P<response_size>\S+) "(?P<referrer>[^\"]+)" "(?P<user_agent>[^\"]+)"'
regex = re.compile(pattern)
log_file = "../Datos/Originales/Redes/apache-logs.txt"
logs = []
with open(log_file, 'r') as file:
    for line in file:
        match = regex.match(line.strip())
        if match:
            logs.append(match.groupdict())
df_apache = pd.DataFrame(logs)

# Firewall
df_firewall = pd.read_csv("../Datos/Originales/Redes/Firewall_logs.csv")
columnas = ['Source_Port', 'Destination_Port', 'NAT_Source_Port',
       'NAT_Destination_Port', 'Action', 'Bytes', 'Bytes_Sent',
       'Bytes_Received', 'Packets', 'Elapsed_Time_(sec)', 'pkts_sent',
       'pkts_received']
df_firewall.columns = columnas

# Network
df_network = pd.read_csv("../Datos/Originales/Redes/Network_flows.csv")
df_network_corto = df_network[0:10000]
df_network_corto.columns = [col.replace('.', '_') for col in df_network_corto.columns]

## 4. Insertar datasets en MySQL

In [13]:
engine = create_engine(f"mysql+pymysql://{usuario}:{contrasena}@{ruta}/{database}")

# Compresores
df_compresores = compresores.copy()
df_compresores.to_sql(con=engine, name='compresores', if_exists='replace', chunksize=10000, index=False)

# Apache
df_apache.to_sql(con=engine, name='apache', if_exists='replace', chunksize=10000, index=False)

# Firewall
df_firewall.to_sql(con=engine, name='firewall', if_exists='replace', chunksize=10000, index=False)

# Network
df_network_corto.to_sql(con=engine, name='network', if_exists='replace', chunksize=10000, index=False)

10000

## 5. Consultas de MongoDB con los DataFrames cargados
A continuación se muestran ejemplos de cómo insertar y consultar los datos de los DataFrames en MongoDB usando `pymongo`.

In [14]:
# Conexión a MongoDB (ajusta el host/puerto si es necesario)
mongo_client = MongoClient('mongodb://localhost:27017/')
mongo_db = mongo_client['reto8_azul_claro']

In [15]:
# Insertar los DataFrames en colecciones de MongoDB
def insert_df_to_mongo(df, collection_name):
    records = df.to_dict(orient='records')
    mongo_db[collection_name].delete_many({})  # Limpia la colección antes de insertar
    if records:
        mongo_db[collection_name].insert_many(records)

insert_df_to_mongo(df_compresores, 'compresores')
insert_df_to_mongo(df_apache, 'apache')
insert_df_to_mongo(df_firewall, 'firewall')
insert_df_to_mongo(df_network_corto, 'network')

### Ejemplo de consultas en MongoDB

In [16]:
# Consulta 1: Obtener datos de los primeros 5 compresores
for doc in mongo_db['compresores'].find().limit(5):
    print(doc)

{'_id': ObjectId('6839e23c512e177ac61fc050'), 'Presion': 0.775747873, 'Temperatura': 16.9, 'Frecuencia': 0.0, 'Potencia_Medida': 71.46656241, 'Potencia_Estimada': 77.43255618, 'Compresor': 'A'}
{'_id': ObjectId('6839e23c512e177ac61fc051'), 'Presion': 0.776314729, 'Temperatura': 16.6, 'Frecuencia': 0.0, 'Potencia_Medida': 71.44276759, 'Potencia_Estimada': 77.49819406, 'Compresor': 'A'}
{'_id': ObjectId('6839e23c512e177ac61fc052'), 'Presion': 0.776783702, 'Temperatura': 16.6, 'Frecuencia': 0.0, 'Potencia_Medida': 71.3476526, 'Potencia_Estimada': 77.47715131, 'Compresor': 'A'}
{'_id': ObjectId('6839e23c512e177ac61fc053'), 'Presion': 0.777092521, 'Temperatura': 16.4, 'Frecuencia': 0.0, 'Potencia_Medida': 71.24480667, 'Potencia_Estimada': 77.52400974, 'Compresor': 'A'}
{'_id': ObjectId('6839e23c512e177ac61fc054'), 'Presion': 0.777435742, 'Temperatura': 16.4, 'Frecuencia': 0.0, 'Potencia_Medida': 71.19410129, 'Potencia_Estimada': 77.50860945, 'Compresor': 'A'}


In [17]:
# Consulta 2: Contar cuántos logs de apache tienen status 404
count_404 = mongo_db['apache'].count_documents({'http_status': '404'})
print(f"Logs con status 404: {count_404}")

Logs con status 404: 2842


In [18]:
# Consulta 3: Obtener los 3 primeros registros de firewall donde Action sea 'deny'
for doc in mongo_db['firewall'].find({'Action': 'deny'}).limit(3):
    print(doc)

{'_id': ObjectId('6839e23f512e177ac622a897'), 'Source_Port': 13394, 'Destination_Port': 23, 'NAT_Source_Port': 0, 'NAT_Destination_Port': 0, 'Action': 'deny', 'Bytes': 60, 'Bytes_Sent': 60, 'Bytes_Received': 0, 'Packets': 1, 'Elapsed_Time_(sec)': 0, 'pkts_sent': 1, 'pkts_received': 0}
{'_id': ObjectId('6839e23f512e177ac622a898'), 'Source_Port': 61078, 'Destination_Port': 57470, 'NAT_Source_Port': 0, 'NAT_Destination_Port': 0, 'Action': 'deny', 'Bytes': 62, 'Bytes_Sent': 62, 'Bytes_Received': 0, 'Packets': 1, 'Elapsed_Time_(sec)': 0, 'pkts_sent': 1, 'pkts_received': 0}
{'_id': ObjectId('6839e23f512e177ac622a8a4'), 'Source_Port': 62776, 'Destination_Port': 62413, 'NAT_Source_Port': 0, 'NAT_Destination_Port': 0, 'Action': 'deny', 'Bytes': 146, 'Bytes_Sent': 146, 'Bytes_Received': 0, 'Packets': 1, 'Elapsed_Time_(sec)': 0, 'pkts_sent': 1, 'pkts_received': 0}


In [19]:
# Consulta 4: Obtener los compresores con Temperatura mayor a 30 y ordenados por Presion descendente
for doc in mongo_db['compresores'].find({'Temperatura': {'$gt': 30}}).sort('Presion', -1):
    print(doc)

{'_id': ObjectId('6839e23c512e177ac6204267'), 'Presion': 0.787148338, 'Temperatura': 30.2, 'Frecuencia': 0.0, 'Potencia_Medida': 73.11542433, 'Potencia_Estimada': 72.88346487, 'Compresor': 'A'}
{'_id': ObjectId('6839e23c512e177ac6204268'), 'Presion': 0.786602369, 'Temperatura': 30.2, 'Frecuencia': 0.0, 'Potencia_Medida': 73.10884963, 'Potencia_Estimada': 72.90796243, 'Compresor': 'A'}
{'_id': ObjectId('6839e23c512e177ac61ff48b'), 'Presion': 0.78588702, 'Temperatura': 30.3, 'Frecuencia': 0.0, 'Potencia_Medida': 72.4063624, 'Potencia_Estimada': 72.90970253, 'Compresor': 'A'}
{'_id': ObjectId('6839e23c512e177ac6204269'), 'Presion': 0.784338048, 'Temperatura': 31.0, 'Frecuencia': 0.0, 'Potencia_Medida': 73.17271095, 'Potencia_Estimada': 72.76670188, 'Compresor': 'A'}
{'_id': ObjectId('6839e23c512e177ac61ff48c'), 'Presion': 0.783702493, 'Temperatura': 30.3, 'Frecuencia': 0.0, 'Potencia_Medida': 72.61410131, 'Potencia_Estimada': 73.007722, 'Compresor': 'A'}
{'_id': ObjectId('6839e23c512e177a

In [20]:
# Consulta 5: Contar cuántos logs de apache tienen método GET y status 200
count_get_200 = mongo_db['apache'].count_documents({'http_method': 'GET', 'http_status': '200'})
print(f"Logs GET con status 200: {count_get_200}")

Logs GET con status 200: 3808


In [21]:
# Consulta 6: Obtener los 5 registros de firewall con mayor cantidad de Bytes enviados
for doc in mongo_db['firewall'].find().sort('Bytes_Sent', -1).limit(5):
    print(doc)

{'_id': ObjectId('6839e23f512e177ac622cff4'), 'Source_Port': 57235, 'Destination_Port': 15187, 'NAT_Source_Port': 23276, 'NAT_Destination_Port': 15187, 'Action': 'allow', 'Bytes': 1269359015, 'Bytes_Sent': 948477220, 'Bytes_Received': 320881795, 'Packets': 1036116, 'Elapsed_Time_(sec)': 9283, 'pkts_sent': 747520, 'pkts_received': 288596}
{'_id': ObjectId('6839e240512e177ac62397fd'), 'Source_Port': 15792, 'Destination_Port': 3478, 'NAT_Source_Port': 30536, 'NAT_Destination_Port': 3478, 'Action': 'allow', 'Bytes': 428935914, 'Bytes_Sent': 213443641, 'Bytes_Received': 215492273, 'Packets': 635946, 'Elapsed_Time_(sec)': 2242, 'pkts_sent': 308738, 'pkts_received': 327208}
{'_id': ObjectId('6839e240512e177ac6232cb7'), 'Source_Port': 15503, 'Destination_Port': 62336, 'NAT_Source_Port': 46736, 'NAT_Destination_Port': 62336, 'Action': 'allow', 'Bytes': 127653507, 'Bytes_Sent': 122661116, 'Bytes_Received': 4992391, 'Packets': 161030, 'Elapsed_Time_(sec)': 2162, 'pkts_sent': 82907, 'pkts_received

In [22]:
# Consulta 7: Obtener los registros de network donde Source_Port sea 80 o 443 y Bytes > 5000
for doc in mongo_db['network'].find({'Source_Port': {'$in': [80, 443]}, 'Bytes': {'$gt': 5000}}):
    print(doc)

In [23]:
# Consulta 8: Agrupar logs de apache por http_status y contar cuántos hay de cada uno
from collections import Counter
status_counts = Counter([doc['http_status'] for doc in mongo_db['apache'].find({}, {'http_status': 1})])
print(dict(status_counts))

{'200': 5619, '404': 2842, '302': 998, '405': 12, '500': 712, '303': 3407, '301': 18, '403': 3, '400': 11}


In [24]:
# Consulta 9: Obtener los 3 registros de compresores con mayor Caudal si existe ese campo
if 'Caudal' in df_compresores.columns:
    for doc in mongo_db['compresores'].find().sort('Caudal', -1).limit(3):
        print(doc)
else:
    print("La columna 'Caudal' no existe en compresores.")

La columna 'Caudal' no existe en compresores.


In [25]:
# Consulta 10: Obtener los registros de firewall donde Action sea 'allow' y Bytes_Received > 2000
for doc in mongo_db['firewall'].find({'Action': 'allow', 'Bytes_Received': {'$gt': 2000}}):
    print(doc)

{'_id': ObjectId('6839e23f512e177ac622a809'), 'Source_Port': 56258, 'Destination_Port': 3389, 'NAT_Source_Port': 56258, 'NAT_Destination_Port': 3389, 'Action': 'allow', 'Bytes': 4768, 'Bytes_Sent': 1600, 'Bytes_Received': 3168, 'Packets': 19, 'Elapsed_Time_(sec)': 17, 'pkts_sent': 10, 'pkts_received': 9}
{'_id': ObjectId('6839e23f512e177ac622a80c'), 'Source_Port': 50002, 'Destination_Port': 443, 'NAT_Source_Port': 45848, 'NAT_Destination_Port': 443, 'Action': 'allow', 'Bytes': 25358, 'Bytes_Sent': 6778, 'Bytes_Received': 18580, 'Packets': 31, 'Elapsed_Time_(sec)': 16, 'pkts_sent': 13, 'pkts_received': 18}
{'_id': ObjectId('6839e23f512e177ac622a80d'), 'Source_Port': 51465, 'Destination_Port': 443, 'NAT_Source_Port': 39975, 'NAT_Destination_Port': 443, 'Action': 'allow', 'Bytes': 3961, 'Bytes_Sent': 1595, 'Bytes_Received': 2366, 'Packets': 21, 'Elapsed_Time_(sec)': 16, 'pkts_sent': 12, 'pkts_received': 9}
{'_id': ObjectId('6839e23f512e177ac622a80f'), 'Source_Port': 50049, 'Destination_Po


{'_id': ObjectId('6839e23f512e177ac623066f'), 'Source_Port': 39803, 'Destination_Port': 443, 'NAT_Source_Port': 33160, 'NAT_Destination_Port': 443, 'Action': 'allow', 'Bytes': 1701571, 'Bytes_Sent': 69958, 'Bytes_Received': 1631613, 'Packets': 1930, 'Elapsed_Time_(sec)': 170, 'pkts_sent': 782, 'pkts_received': 1148}
{'_id': ObjectId('6839e23f512e177ac6230672'), 'Source_Port': 38873, 'Destination_Port': 443, 'NAT_Source_Port': 28510, 'NAT_Destination_Port': 443, 'Action': 'allow', 'Bytes': 10209, 'Bytes_Sent': 2574, 'Bytes_Received': 7635, 'Packets': 39, 'Elapsed_Time_(sec)': 917, 'pkts_sent': 18, 'pkts_received': 21}
{'_id': ObjectId('6839e23f512e177ac623067a'), 'Source_Port': 50432, 'Destination_Port': 5222, 'NAT_Source_Port': 28391, 'NAT_Destination_Port': 5222, 'Action': 'allow', 'Bytes': 28281, 'Bytes_Sent': 9813, 'Bytes_Received': 18468, 'Packets': 196, 'Elapsed_Time_(sec)': 608, 'pkts_sent': 100, 'pkts_received': 96}
{'_id': ObjectId('6839e23f512e177ac623067e'), 'Source_Port': 5

In [26]:
# Consulta 11: Contar cuántos registros de network tienen pkts_sent > 100 y pkts_received > 100
count_pkts = mongo_db['network'].count_documents({'pkts_sent': {'$gt': 100}, 'pkts_received': {'$gt': 100}})
print(f"Registros de network con pkts_sent > 100 y pkts_received > 100: {count_pkts}")

Registros de network con pkts_sent > 100 y pkts_received > 100: 0


In [27]:
# Consulta 12: Aggregate - Promedio de Bytes por Action en firewall
pipeline = [
    {"$group": {"_id": "$Action", "avg_bytes": {"$avg": "$Bytes"}}},
    {"$sort": {"avg_bytes": -1}}
]
for doc in mongo_db['firewall'].aggregate(pipeline):
    print(doc)

{'_id': 'allow', 'avg_bytes': 169037.95324123272}
{'_id': 'reset-both', 'avg_bytes': 157.35185185185185}
{'_id': 'deny', 'avg_bytes': 83.21171682124508}
{'_id': 'drop', 'avg_bytes': 68.67527818846783}


In [28]:
# Consulta 13: Aggregate - Contar cuántos registros de network hay por cada valor de Source_Port (top 5)
pipeline = [
    {"$group": {"_id": "$Source_Port", "count": {"$sum": 1}}},
    {"$sort": {"count": -1}},
    {"$limit": 5}
]
for doc in mongo_db['network'].aggregate(pipeline):
    print(doc)

{'_id': 3128, 'count': 3477}
{'_id': 443, 'count': 227}
{'_id': 80, 'count': 47}
{'_id': 5228, 'count': 9}
{'_id': 51433, 'count': 8}
